# NLP Project

- In our last exploring NLP notebook we built an email spam detector using Natural Language Processing techniques and the Support Vector Machine (SVM) algorithm for classification.

- In this project, we will again build a spam detector but this time using URLs instead of emails.

In [1]:
%pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.1.2
    Uninstalling pip-22.1.2:
      Successfully uninstalled pip-22.1.2
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -r ../requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 111.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 125.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 99.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 24.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 130.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 155.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 152.2 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.6/156.6 kB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [9]:
!pip install nltk
import pandas as pd 
import numpy as np
import nltk
import re
import unicodedata
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
nltk.download('stopwords')

  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
  Using cached regex-2022.7.25-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (768 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)


[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
df_raw = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/machine-learning-content/master/assets/spam.csv")

In [11]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [12]:
df_raw.sample(5)

,Category,Message
1029,ham,"Lol you forgot it eh ? Yes, I'll bring it in babe"
5316,ham,Jus finish watching tv... U?
5094,ham,"Hi Shanil,Rakhesh here.thanks,i have exchanged..."
64,ham,Ok lar i double check wif da hair dresser alre...
2384,ham,Your pussy is perfect!


In [13]:
# Tratar de llevar la informacion de texto a minuscula
df_raw['Message'] = df_raw['Message'].str.lower()

In [14]:
# Elementos de texto que no son informativos - stopwords
# Tratar de encontrar esos 
df_raw['Message'].str.split(expand=True).stack().value_counts()[:60]

to           2234
i            2217
you          1921
a            1433
the          1326
u             996
and           968
is            868
in            857
my            755
for           704
your          677
of            614
me            611
have          568
call          556
on            521
are           487
that          470
it            466
2             457
so            423
but           422
or            415
not           411
at            400
can           386
ur            385
if            382
with          379
will          379
i'm           377
be            376
get           375
do            364
just          363
we            346
this          309
when          283
from          277
&lt;#&gt;     276
go            265
up            264
all           261
no            258
4             255
how           254
what          252
now           247
.             241
like          236
got           235
know          230
was           230
free          228
out       

In [15]:
# funcion para retirar los stopswords

stop=stopwords.words('english')
def remove_stopwords(message):
  if message is not None:
    words = message.strip().split()
    words_filtered = []
    for word in words:
      if word not in stop:
        words_filtered.append(word) 
    result = " ".join(words_filtered)         
  else:
    result = None

  return result  


In [16]:
df_interin = df_raw.copy()

In [17]:
df_interin['Message'] = df_interin['Message'].apply(remove_stopwords)

In [18]:
df_interin['Message'].str.split(expand=True).stack().value_counts()[:60]

u            996
call         556
2            457
ur           385
i'm          377
get          375
&lt;#&gt;    276
go           265
4            255
.            241
like         236
got          235
know         230
free         228
come         220
good         201
?            187
send         187
want         183
text         175
time         169
i'll         168
...          163
love         163
going        161
ok           160
ü            157
need         157
r            153
still        151
one          150
txt          149
n            146
see          145
dont         140
new          136
tell         135
think        127
reply        126
mobile       124
take         124
back         121
stop         119
please       118
home         112
day          111
&            111
hi           108
claim        104
hope         103
make         101
give         100
pls           99
me.           98
phone         97
now.          94
later         94
happy         93
much          

In [19]:
df_interin['Message'] = df_interin['Message'].str.replace(".","",regex=False)

In [20]:
# primer regex
df_interin['Message'] = df_interin['Message'].str.replace('''[?&#,;ü']''','',regex=True)

In [21]:
df_interin['Message'].str.split(expand=True).stack().value_counts()[:60]

u         1105
call       573
2          472
im         461
ur         386
get        382
go         277
ltgt       276
ok         275
4          269
free       253
know       249
like       243
ill        238
got        237
good       232
come       228
time       205
want       193
send       190
love       189
text       186
day        183
going      170
one        170
me         167
need       166
lor        160
home       160
you        156
see        155
still      154
now        154
sorry      153
r          153
stop       150
back       150
txt        149
dont       147
n          146
reply      143
tell       137
new        136
later      134
think      132
mobile     128
today      128
hi         128
well       127
it         127
take       126
please     125
da         125
cant       124
phone      118
claim      111
night      111
much       109
dear       109
hey        107
dtype: int64

In [22]:
# Funcion adicional
# Para las palabras en español
def normalize_string(message):
  if message is not None:
     result = unicodedata.normalize('NFC',message).encode('ascii','ignore').decode()
  else:
    result = None
  return result  

In [23]:
df_interin['Message']= df_interin['Message'].apply(normalize_string)

In [24]:
# Es comun que escriban goooooddd looooove o etc etc.

def replace_multiple_letters(message):
  if message is not None:
    result = re.sub(r"([a-zA-Z])\1{2,}",r"\1",message)
  else:
    result= None
  return result  

In [25]:
df_interin['Message']= df_interin['Message'].apply(replace_multiple_letters)

In [26]:
df_interin['Message'].str.split(expand=True).stack().value_counts()[:60]

u         1105
call       573
2          472
im         470
ur         386
get        382
go         277
ltgt       276
ok         275
4          269
free       253
know       249
like       243
ill        242
got        237
good       232
come       228
time       205
want       193
love       190
send       190
text       186
day        183
one        170
going      170
me         167
need       166
home       160
lor        160
dont       158
you        157
see        155
now        154
still      154
sorry      153
r          153
stop       150
back       150
txt        149
n          146
reply      143
tell       137
new        136
later      134
think      132
today      128
well       128
hi         128
mobile     128
it         127
cant       126
take       126
please     126
da         125
phone      118
claim      111
night      111
oh         110
much       109
dear       109
dtype: int64

In [27]:
df = df_interin.copy()

In [28]:
X = df['Message']
y = df['Category']
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=121)

In [29]:
# Vectorizador
vec = CountVectorizer(stop_words='english')
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.transform(X_test).toarray()

In [30]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [31]:
X_train.shape
# filas = 4179 mensajes
# columnas = 7559 palabras
# Esto es para el train - faltan los de test

(4179, 7559)

In [32]:
X_test.shape

(1393, 7559)

In [33]:
print(vec.get_feature_names_out())

['008704050406' '0089' '0121' ... 'zoe' 'zogtorius' 'zoom']


In [34]:
nb = MultinomialNB()

In [35]:
nb.fit(X_train, y_train)

MultinomialNB()

In [36]:
nb.score(X_train,y_train)

0.9952141660684374

In [37]:
nb.score(X_test,y_test)

0.9820531227566404

In [38]:
nb.predict(vec.transform(['hello my friend']))

array(['ham'], dtype='<U4')

In [39]:
nb.predict(vec.transform(['free for season']))

array(['spam'], dtype='<U4')

In [40]:
def compress_date_street_name(message):
  if message is not None:
    expr = "(\d{1,2})\sde\s([ene|feb|mar|abr|may|jun|jul|ago|set|sep|oct|nov|dic].*)"
    result = re.sub(expr,r"\1de\2",message)
  else:
    result=None
  return result 

In [41]:
compress_date_street_name("calle 18 de julio N.2354 Localidad Durazno Pais Uruguay")

'calle 18dejulio N.2354 Localidad Durazno Pais Uruguay'